In [1]:
class RandomRotation(object):
    """
    https://github.com/pytorch/vision/tree/master/torchvision/transforms
    Rotate the image by angle.
    Args:
        degrees (sequence or float or int): Range of degrees to select from.
            If degrees is a number instead of sequence like (min, max), the range of degrees
            will be (-degrees, +degrees).
        resample ({PIL.Image.NEAREST, PIL.Image.BILINEAR, PIL.Image.BICUBIC}, optional):
            An optional resampling filter.
            See http://pillow.readthedocs.io/en/3.4.x/handbook/concepts.html#filters
            If omitted, or if the image has mode "1" or "P", it is set to PIL.Image.NEAREST.
        expand (bool, optional): Optional expansion flag.
            If true, expands the output to make it large enough to hold the entire rotated image.
            If false or omitted, make the output image the same size as the input image.
            Note that the expand flag assumes rotation around the center and no translation.
        center (2-tuple, optional): Optional center of rotation.
            Origin is the upper left corner.
            Default is the center of the image.
    """

    def __init__(self, degrees, resample=False, expand=False, center=None):
        if isinstance(degrees, numbers.Number):
            if degrees < 0:
                raise ValueError("If degrees is a single number, it must be positive.")
            self.degrees = (-degrees, degrees)
        else:
            if len(degrees) != 2:
                raise ValueError("If degrees is a sequence, it must be of len 2.")
            self.degrees = degrees

        self.resample = resample
        self.expand = expand
        self.center = center

    @staticmethod
    def get_params(degrees):
        """Get parameters for ``rotate`` for a random rotation.
        Returns:
            sequence: params to be passed to ``rotate`` for random rotation.
        """
        angle = np.random.uniform(degrees[0], degrees[1])

        return angle

    def __call__(self, img):
        """
            img (PIL Image): Image to be rotated.
        Returns:
            PIL Image: Rotated image.
        """
        
        def rotate(img, angle, resample=False, expand=False, center=None):
            """Rotate the image by angle and then (optionally) translate it by (n_columns, n_rows)
            Args:
            img (PIL Image): PIL Image to be rotated.
            angle ({float, int}): In degrees degrees counter clockwise order.
            resample ({PIL.Image.NEAREST, PIL.Image.BILINEAR, PIL.Image.BICUBIC}, optional):
            An optional resampling filter.
            See http://pillow.readthedocs.io/en/3.4.x/handbook/concepts.html#filters
            If omitted, or if the image has mode "1" or "P", it is set to PIL.Image.NEAREST.
            expand (bool, optional): Optional expansion flag.
            If true, expands the output image to make it large enough to hold the entire rotated image.
            If false or omitted, make the output image the same size as the input image.
            Note that the expand flag assumes rotation around the center and no translation.
            center (2-tuple, optional): Optional center of rotation.
            Origin is the upper left corner.
            Default is the center of the image.
            """
                
            return img.rotate(angle, resample, expand, center)

        angle = self.get_params(self.degrees)

        return rotate(img, angle, self.resample, self.expand, self.center)
class RandomShift(object):
    def __init__(self, shift):
        self.shift = shift
        
    @staticmethod
    def get_params(shift):
        """Get parameters for ``rotate`` for a random rotation.
        Returns:
            sequence: params to be passed to ``rotate`` for random rotation.
        """
        hshift, vshift = np.random.uniform(-shift, shift, size=2)

        return hshift, vshift 
    def __call__(self, img):
        hshift, vshift = self.get_params(self.shift)
        
        return img.transform(img.size, Image.AFFINE, (1,0,hshift,0,1,vshift), resample=Image.BICUBIC, fill=1)

In [2]:
from PIL import Image,ImageFile
import io
import os
import pandas as pd
import csv
import sys
import cv2
import numpy as np
import time
size=224
import torch
from torchvision.transforms import ColorJitter, ToTensor, Normalize,Compose,Resize,ToPILImage
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image, ImageOps, ImageEnhance
import numbers
import Augmentor
device = "cpu"
p=Augmentor.Pipeline()
p.zoom(probability=0.4, min_factor=1.1, max_factor=1.5)
p.crop_centre(probability=0.4,percentage_area=0.2)
p.random_brightness(probability=0.4,min_factor=0.7, max_factor=1.5)
p.random_color(probability=0.4, min_factor=0.5, max_factor=2)
p.gaussian_distortion(probability=0.2, grid_width=5, grid_height=5, magnitude=5,corner='bell',method='in')
train_transform = Compose([
    p.torch_transform(),
#    ToPILImage(),
    Resize((size,size)),
    RandomRotation(degrees=12),
    RandomShift(4),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225])
])
val_transform = Compose([
    Resize((size,size)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225])
])
batch_size = 64
lr=3e-4
train_dataset = ImageFolder('d:\Python\cv\\model_data', transform=train_transform, target_transform=None)
val_dataset = ImageFolder('d:\Python\cv\\test_data', transform=val_transform, target_transform=None)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                          drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True,
                        drop_last=False)

import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from senet import se_resnext50_32x4d
class FineTune_Model(nn.Module):
    def __init__(self,num_classes=2):
        super().__init__()
        self.basemodel=se_resnext50_32x4d(pretrained='imagenet', inchannels=3)
        planes=2048
        self.bottleneck_g = nn.BatchNorm1d(planes)
        self.bottleneck_g.bias.requires_grad_(False)
        self.dropout=nn.Dropout(0.2)
        self.fc = nn.Linear(planes, num_classes)
        nn.init.normal_(self.fc.weight, std=0.001)
        nn.init.constant_(self.fc.bias, 0)
    def forward(self,x):
        x=self.basemodel(x)
        x=F.avg_pool2d(x,x.size()[2:])
        x=x.view(x.size(0),-1)
        x=self.dropout(x)
        x=self.bottleneck_g(x)
        x=self.fc(x)
        return x
    def freeze(self):
        for param in self.basemodel.parameters():
            param.requires_grad = False
        for param in self.basemodel.layer4.parameters():
            param.requires_grad = True
model=FineTune_Model()
model.basemodel.requires_grad=False
model.basemodel.layer4.requires_grad=True
model.load_state_dict(torch.load('d:\Python\\cv\\pytorch_model\\senet_model_train-2finish.pt'))
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth" to C:\Users\Toshevikov/.torch\models\se_resnext50_32x4d-a260b3a4.pth


In [3]:
epochs = 10
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []
train_accuracy=0
best_train=0
best_val=0
model.freeze()
for epoch in range(epochs):
    for inputs, labels in train_loader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        train_accuracy +=torch.mean(equals.type(torch.FloatTensor)).item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device),labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy +=torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(train_loader))
            test_losses.append(test_loss/len(val_loader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Train accuracy: {train_accuracy/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(val_loader):.3f}.. "
                  f"Test accuracy: {accuracy/len(val_loader):.3f}")
            if train_accuracy>=best_train and accuracy>=best_val:
                best_train=train_accuracy
                best_val=accuracy
                torch.save(model.state_dict(),f'd:\Python\\cv\\pytorch_model\\senet_model_train-{train_accuracy/print_every:.3f}_test-{accuracy/len(val_loader):.3f}.pt')
            running_loss = 0
            train_accuracy=0
            model.train()

Epoch 1/10.. Train loss: 0.349.. Train accuracy: 0.866.. Test loss: 0.311.. Test accuracy: 0.875
Epoch 1/10.. Train loss: 0.348.. Train accuracy: 0.847.. Test loss: 0.350.. Test accuracy: 0.862
Epoch 1/10.. Train loss: 0.349.. Train accuracy: 0.869.. Test loss: 0.353.. Test accuracy: 0.865
Epoch 1/10.. Train loss: 0.382.. Train accuracy: 0.858.. Test loss: 0.442.. Test accuracy: 0.789
Epoch 1/10.. Train loss: 0.320.. Train accuracy: 0.864.. Test loss: 0.524.. Test accuracy: 0.815
Epoch 1/10.. Train loss: 0.363.. Train accuracy: 0.836.. Test loss: 0.352.. Test accuracy: 0.837
Epoch 1/10.. Train loss: 0.341.. Train accuracy: 0.853.. Test loss: 0.323.. Test accuracy: 0.859
Epoch 1/10.. Train loss: 0.254.. Train accuracy: 0.895.. Test loss: 0.340.. Test accuracy: 0.861
Epoch 1/10.. Train loss: 0.402.. Train accuracy: 0.836.. Test loss: 0.468.. Test accuracy: 0.797
Epoch 1/10.. Train loss: 0.339.. Train accuracy: 0.858.. Test loss: 0.432.. Test accuracy: 0.813
Epoch 1/10.. Train loss: 0.357

KeyboardInterrupt: 

In [6]:
#torch.save(model.state_dict(),f'd:\Python\\cv\\pytorch_model\\senet_model_train-3finish.pt')
